In [83]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import psutil
import geopandas as gpd
from shapely.geometry import Point

In [84]:
sf2009 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2009singlefamily_final_with_sqft.txt')
sf2013 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2013singlefamily_final_with_sqft.txt')
sf2017 = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/2017singlefamily_final_with_sqft.txt')

In [ ]:
sf2009.head()

In [85]:
sf2009.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2009 LAND':'LandValue', '2009 IMPR':'ImprovementValue', '2009 TOTAL APPR':'AppraisedValue', '2009 TOTAL ASSD':'2009 TOTAL ASSD'}, inplace=True)

In [86]:
sf2013.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2013 LAND':'LandValue', '2013 IMPR':'ImprovementValue', '2013 TOTAL APPR':'AppraisedValue', '2013 TOTAL ASSD':'2013 TOTAL ASSD'}, inplace=True)

In [87]:
sf2017.rename(columns={'APN':'APN', 'DistrictCode':'DistrictCode', 'Council District':'DistrictNo', 'AddressFullAddress': 'Address', 'AddressCity':'City', 'AddressPostalCode':'Zip', '2017 LAND':'LandValue', '2017 IMPR':'ImprovementValue', '2017 TOTAL APPR':'AppraisedValue', '2017 TOTAL ASSD':'2017 TOTAL ASSD'}, inplace=True)

In [ ]:
sf2009.info()

In [ ]:
sf2013.info()

In [ ]:
sf2017.info()

In [ ]:
sf2009.AppraisedValue.describe()

In [ ]:
sf2009.AppraisedValue.plot()

In [ ]:
sf2013.AppraisedValue.describe()

In [ ]:
sf2013.AppraisedValue.plot()

In [ ]:
sf2017.AppraisedValue.describe()

In [ ]:
sf2017.AppraisedValue.plot()

In [ ]:
sf2009.AppraisedValue.describe()

In [88]:
issued = pd.read_csv('/Users/davidhaines/DataScience/data-question-4-affordable-housing-plaid-platypi-plaidypi/metrodata/Building_Permits_Issued.csv', low_memory=False)

In [ ]:
issued.head()

In [89]:
issued.rename(columns={'Permit #': 'PermitNo', 'Permit Type Description':'PermTypeDesc', 'Permit Subtype Description': 'PermSubDesc', 'Parcel':'Parcel', 'Date Entered':'EnteredDt', 'Date Issued': 'IssuedDt', 'Const. Cost': 'ConstCost', 'Address':'Address', 'City':'City', 'State':'State', 'Zip':'Zip', 'Subdivision/Lot': 'Subdiv_Lot', 'Contact':'Contact', 'Permit Type': 'PermType', 'Permit Subtype': 'PermSubtype', 'IVR Trk#': 'IVRTrk#', 'Purpose':'Purpose', 'Council Dist':'CouncilDist', 'Census Tract':'CensusTract', 'Mapped Location':'MappedLoc'}, inplace=True)

In [90]:
issued['EnteredDt'] = pd.to_datetime(issued['EnteredDt'])
issued['IssuedDt'] = pd.to_datetime(issued['IssuedDt'])

In [97]:
issued.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36221 entries, 0 to 36220
Data columns (total 20 columns):
PermitNo        36221 non-null object
PermTypeDesc    36221 non-null object
PermSubDesc     36221 non-null object
Parcel          36221 non-null object
EnteredDt       36221 non-null datetime64[ns]
IssuedDt        36221 non-null datetime64[ns]
ConstCost       36085 non-null float64
Address         36221 non-null object
City            36093 non-null object
State           36096 non-null object
Zip             36221 non-null int64
Subdiv_Lot      36220 non-null object
Contact         36221 non-null object
PermType        36221 non-null object
PermSubtype     36221 non-null object
IVRTrk#         36221 non-null int64
Purpose         35922 non-null object
CouncilDist     36141 non-null float64
CensusTract     36039 non-null float64
MappedLoc       36221 non-null object
dtypes: datetime64[ns](2), float64(3), int64(2), object(13)
memory usage: 5.5+ MB


In [110]:
issued.MappedLoc.unique()

array(['1017A MONROE ST\nNASHVILLE, TN 37208\n(36.174077, -86.797183)',
       '2300 WEST END AVE\nNASHVILLE, TN 37203\n(36.148736, -86.804731)',
       '2802 NOLENSVILLE PIKE\nNASHVILLE, TN 37211\n(36.112749, -86.746151)',
       ...,
       '915 SILKWOOD CIR\nNASHVILLE, TN 37221\n(36.048586, -86.951711)',
       '3562 PIN HOOK RD 301-316\nANTIOCH, TN 37013\n(36.050335, -86.608639)',
       '2007B LOMBARDY AVE\nNASHVILLE, TN 37215\n(36.12021, -86.807099)'],
      dtype=object)

In [ ]:
issued.columns

In [ ]:
issued.PermTypeDesc.unique()

In [ ]:
issued.head()

In [ ]:
permitdecode = pd.DataFrame(issued[['PermTypeDesc','PermType']])

In [ ]:
result = re.findall('-?\d*\.\d*', issued.MappedLoc[1])

In [ ]:
print(result)

In [ ]:
permitdecode.drop_duplicates()

CARN, CARR, CADM

In [111]:
#resnew = pd.DataFrame(issued.loc[issued['PermType'] == 'CARW'])
respermits = pd.DataFrame(issued.loc[issued['PermType'].isin(['CARN','CARR'])])

In [116]:
respermits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17034 entries, 0 to 36220
Data columns (total 20 columns):
PermitNo        17034 non-null object
PermTypeDesc    17034 non-null object
PermSubDesc     17034 non-null object
Parcel          17034 non-null object
EnteredDt       17034 non-null datetime64[ns]
IssuedDt        17034 non-null datetime64[ns]
ConstCost       17029 non-null float64
Address         17034 non-null object
City            16964 non-null object
State           16965 non-null object
Zip             17034 non-null int64
Subdiv_Lot      17034 non-null object
Contact         17034 non-null object
PermType        17034 non-null object
PermSubtype     17034 non-null object
IVRTrk#         17034 non-null int64
Purpose         17031 non-null object
CouncilDist     17015 non-null float64
CensusTract     16954 non-null float64
MappedLoc       17034 non-null object
dtypes: datetime64[ns](2), float64(3), int64(2), object(13)
memory usage: 2.7+ MB


In [117]:
newhomes = respermits.Purpose.str.match(r'[uU]nit')

In [118]:
len(newhomes)

17034

In [112]:
respermits.Purpose.unique()

array(["to construct 1830SF single family residence with 80Sf porches. 5' min. side setback, 20' min. rear setback, front setback at 24.6' per avg. unit 1 of 2. lot coverage is 1173SF of max 3775SF lot coverage. not to be over any easements. Max height 3 stories or 45�.\nSidewalks are required, applicant may pay in lieu of sidewalk requirements\n***PURSUANT TO ORDINANCE NO 2008-1263 of the Metropolitan Code of Laws, I , holder of this permit, hereby certify that all construction and demolition waste generated by any and all activities governed by this permit shall be disposed of in an approved landfill.  Further, I certify that no construction or demolition waste shall be stored on the property in violation of any provisions of the Metropolitan Code* For every 30 feet of street frontage, or fraction thereof, one 2 inch caliper tree as listed in the Urban Forestry approved tree list shall be planted on the subject property.*",
       'to conduct interior renovation to existing unit. no 

In [100]:
demos = pd.DataFrame(issued.loc[issued['PermType'] == 'CADM'])

In [107]:
demos.head()

,PermitNo,PermTypeDesc,PermSubDesc,Parcel,EnteredDt,IssuedDt,ConstCost,Address,City,State,Zip,Subdiv_Lot,Contact,PermType,PermSubtype,IVRTrk#,Purpose,CouncilDist,CensusTract,MappedLoc
23,201543896,Building Demolition Permit,Demolition Permit - Residential,10509038300,2015-10-08,2015-10-08,4000.0,914 ACKLEN AVE,NASHVILLE,TN,37203,LOT 2 WAVERLY PLACE RESUB PT. LOT 31,Graham Construction Of Tennessee Llc,CADM,CAZ01A001,2075785,Work on this demolition permit must commence w...,17.0,37016200.0,"914 ACKLEN AVE\nNASHVILLE, TN 37203\n(36.13354..."
30,201620413,Building Demolition Permit,Demolition Permit - Residential,08116022300,2016-05-03,2016-05-03,5000.0,918 SCOVEL ST,NASHVILLE,TN,37208,LOT 55 MCGAVOCK TOWN NORTH NASHVILLE,K Jordan Construction Llc,CADM,CAZ01A001,2120019,Work on this demolition permit must commence w...,19.0,37019400.0,"918 SCOVEL ST\nNASHVILLE, TN 37208\n(36.173632..."
36,201614474,Building Demolition Permit,Demolition Permit - Residential,13102006500,2016-03-31,2016-03-31,7500.0,1923 WARFIELD DR,NASHVILLE,TN,37215,LOT 21 COLONIAL HGTS,Cat3 Restoration Llc,CADM,CAZ01A001,2112314,to demolish exisitng single family residence. ...,25.0,37017901.0,"1923 WARFIELD DR\nNASHVILLE, TN 37215\n(36.102..."
37,2016039565,Building Demolition Permit,Demolition Permit - Residential,10200007600,2016-08-03,2016-08-03,21000.0,3985 OLD HICKORY BLVD,NASHVILLE,TN,37218,S/S OF OLD HICKORY BV E OF HYDES FERRY PK,SELF CONTRACTOR COMMERCIAL (SEE APPLICANT INFO...,CADM,CAZ01A001,3264459,to demolish existing single family residence. ...,1.0,37013100.0,"3985 OLD HICKORY BLVD\nNASHVILLE, TN 37218\n(3..."
47,2017033484,Building Demolition Permit,Demolition Permit - Residential,15900008000,2017-05-19,2017-05-19,10000.0,5610 GRANNY WHITE PIKE,BRENTWOOD,TN,37027,LOTS 1 & 2 GRANNY WHITE PARTNERS,"URBAN DEVELOPMENT GROUP, LLC",CADM,CAZ01A001,3368029,to demolish existing residence. not to be bur...,34.0,37018700.0,"5610 GRANNY WHITE PIKE\nBRENTWOOD, TN 37027\n(..."


In [92]:
respermits.set_index('IssuedDt', inplace=True)

In [99]:
respermits.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17034 entries, 0 to 36220
Data columns (total 20 columns):
PermitNo        17034 non-null object
PermTypeDesc    17034 non-null object
PermSubDesc     17034 non-null object
Parcel          17034 non-null object
EnteredDt       17034 non-null datetime64[ns]
IssuedDt        17034 non-null datetime64[ns]
ConstCost       17029 non-null float64
Address         17034 non-null object
City            16964 non-null object
State           16965 non-null object
Zip             17034 non-null int64
Subdiv_Lot      17034 non-null object
Contact         17034 non-null object
PermType        17034 non-null object
PermSubtype     17034 non-null object
IVRTrk#         17034 non-null int64
Purpose         17031 non-null object
CouncilDist     17015 non-null float64
CensusTract     16954 non-null float64
MappedLoc       17034 non-null object
dtypes: datetime64[ns](2), float64(3), int64(2), object(13)
memory usage: 2.7+ MB


In [96]:
len(respermits.PermSubDesc.unique())

28

In [ ]:
monthlycost = respermits['ConstCost'].resample('M').mean()

In [ ]:
monthlycost.plot()

In [ ]:
respermits.describe()

In [ ]:
resnew.EnteredDt.describe()

In [ ]:
resnew.IssuedDt.describe()

In [ ]:
resnew.ConstCost.describe()

In [ ]:
issued['CouncilDist'].unique()

In [ ]:
issued['MappedLoc'][1]

In [ ]:
issued['extcoords'] = issued.MappedLoc.str.extract(r'?\(.*\)', expand=False)

In [ ]:
coords = re.compile(r'\(.*\)')

In [ ]:
matches = []

In [ ]:
for string in issued.MappedLoc:
    match = re.findall(coords, string)
    matches.append(match)

In [ ]:
type(matches)

In [ ]:
type(matches[1])

In [ ]:
#[s for s in i[s] for i in matches]

coordlist = []
for i in enumerate(matches):
    i = []
    for s in i[s]:
        coordlist.append(tuple(s))

In [ ]:
len(matches)

In [ ]:
matches[1]

In [ ]:
issued['coords'] = pd.Series(matches).astype(tuple)

In [ ]:
issued['coords'][1]

Income data:
    Median average income: 68000
        Income subgroups:
        30% MAI (20400 /yr) (30% housing = 510/mo)
        60% MAI (40800 /yr) (30% housing = 1020/mo)
        80% MAI (54400 /yr) (30% housing = 1360/mo)
        120% MAI (81600 /yr) (30% housing = 2040/mo)

Model development:
    
    Owners = 54%
        - Model annual adjustments to these numbers
    
    
    Median Housing Cost - Owner = 1336/month * 12 = 16032/yr

    15-yr owner gap (affordable) = 3367
    15-yr owner gap (market) = 35751

    10-yr predicted owner gap (affordable) = 1257
    10-yr predicted owner gap (market) = 41421
    

In [ ]:
issued.Issued.min()

In [ ]:
issued.Issued.max()

In [ ]:
issued['monthlies'] = pd.Series([cost/360 for cost in issued.ConstCost])

In [ ]:
issued.info()

In [ ]:
issued.Zip.unique()

Question 1: At what point in time did the Codes Administration decide to begin “closing the loophole” by converting R zoning to RS?

Response 1: The Codes Administration does not set zoning policy, we interpret/enforce it. The Planning Commission and Metro Council are the ones who change zoning rules. The last major revision countywide to the zoning code was in 1997 when they rezoned a lot of properties in bulk. Since then, Councilmembers have proposed zoning changes for their neighborhoods on a piecemeal basis, or developers have requested zoning changes for particular properties. There hasn’t been an effort by Metro Government to downzone properties because of the duplex situation, but some Council members have. (edited)

Question 2: According to the report (Housing Nashville), there are predetermined amounts of needed housing and rental properties in each district. How is the split between housing demand and rental demand being evaluated? Is this arbitrary or is there census data giving us a proportion to go off of?

Question 3: What criteria was used to calculated the ‘Maximum Affordable Sales Price’  from the MHI on page 56 of the Housing Nashville Report? What interest rate and/or tax rate was assumed?

Response 2 & 3: Unfortunately, the folks that produced the Housing Nashville Report have left Metro Government so I don’t know that we’ll get all the answers. Looking at question three, I believe the estimate was based on a 5% down payment and about a 4% interest rate for a 30 year mortgage.
Also, We have also asked Assessments for an update to the 3-years of assessment data to include the square footage of the property and the zoning code for the property. (edited)

Renters = 46%
Median Housing Cost - Renter = 874/month * 12 = 10488/yr
    
    15-yr rental gap (affordable) = 17754
    15-yr rental gap (market) = 22219
        
    10-yr predicted rental gap (affordable) = 30934
    10-yr predicted rental gap (market) = 23043